<a href="https://colab.research.google.com/github/mbagci06/lung_covid/blob/main/DeepLab_train_lung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# BASICS 
import os
import numpy as np
import torch

# Colab 
from pydrive.drive import GoogleDrive
from google.colab import drive
from google.colab import files
from pydrive.auth import GoogleAuth
from google.colab import auth
import torchvision.transforms as transforms
from oauth2client.client import GoogleCredentials
drive.mount('/content/drive')
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# MODEL and TRAIN
import torchvision
import torchvision.models
from torch.utils.data import DataLoader,Dataset
import random
from skimage import io
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import pyplot as plt
from torch import optim
from PIL import Image

Mounted at /content/drive


In [ ]:
# the functions have called with network of the deeplab 
!cp /content/drive/Shareddrives/FurkanDataandModels/Furkan_covid/codes/functions.ipynb  /content
!cp -R /content/drive/Shareddrives/FurkanDataandModels/Furkan_covid/codes/networks /content

In [ ]:
# Run the function code to call the funcctions in it
%run functions.ipynb
# Call the library of the network with the deeplab versions
from networks import deeplab_xception, deeplab_resnet

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# The Calling the validaton dataset
val=Lung_Dataset_deeplab(split = "Val",class_folder='lung',shuffle=True,sample_size=300)# The length of the validation dataset to calculate the average value of the loss and accuracy

val_size=val.__len__()
# The Train dataset 
tra=Lung_Dataset_deeplab(split = "Train",class_folder='lung',shuffle=True,sample_size=2000)
tra_size=tra.__len__()

# Call cuda, if there is no GPU use CPU
device=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# The input channel determined already because model is pretrained 
net = deeplab_resnet.DeepLabv3_plus(nInputChannels=3, n_classes=2, os=8, pretrained=True).to(device=device)
# The adam optimizer for optimization the values will be same for the other model training
optimizer = optim.Adam(net.parameters(), lr= 0.0001)
# The Loss function Cross entropy the value 
# BCE logistic do not need to normalization to 0 1 
criterion = nn.CrossEntropyLoss()

Constructing DeepLabv3+ model...
Number of classes: 2
Output stride: 8
Number of Input Channels: 3


Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth


  0%|          | 0.00/170M [00:00<?, ?B/s]

In [ ]:
batch_size=25
# loading images and masks for mini epochs 
data_loader = torch.utils.data.DataLoader(
        tra, batch_size=batch_size,  num_workers=2, drop_last=True)

data_loader_val = torch.utils.data.DataLoader(
        val, batch_size=batch_size,  num_workers=2, drop_last=True)

In [ ]:
!nvidia-smi

Mon May 30 04:59:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    34W / 250W |   1173MiB / 16280MiB |      5%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# The number of the epoch 
epoch_number=20
# list for the calculating validation Loss
val_loss_array=[]
# list for the calculating train Loss
train_loss_array=[]
# list for the calculating validation accuracy
acc_val_array=[]
# list for the calculating train accuracy
acc_train_array=[]
for t in range(epoch_number):
    # reset the loss and the accuracy values each epoch 
    val_loss=0
    train_loss=0
    acc_val=0
    acc_train=0
    for batch_idx, sample in enumerate(data_loader):
        # Train data preparing 
        imgs , true_masks = sample['image'],sample['mask']
        imgs = imgs.to(device=device)
        true_masks=Negative_mask(true_masks,imgs.size(dim=0))
        true_masks = true_masks.to(device=device)
        net.train()
        # zero the parameter gradients
        optimizer.zero_grad()
        

        with torch.set_grad_enabled(True):
            # Train data evaluation
            masks_pred = net(imgs)
            loss = criterion(masks_pred, true_masks)
            train_loss += loss.item()
            # Updating the params
            loss.backward()
            optimizer.step()
            # Acc calc train
            acc_train+=dice( masks_pred, true_masks)
    for batch_idx, sample in enumerate(data_loader_val):
        # Validation data preparing  and calling 
        # calcel gradient feature 
        net.eval()

        # Call iamge and mask
        imgs , true_masks = sample['image'],sample['mask']

        imgs = imgs.to(device=device)
        true_masks=Negative_mask(true_masks,imgs.size(dim=0))
        true_masks = true_masks.to(device=device)

        # zero the parameter gradients
        optimizer.zero_grad()

        with torch.set_grad_enabled(False):
            # Validation evaluation 
            masks_pred = net(imgs)
            masks_pred = masks_pred.to(device=device)
            loss_val = criterion(masks_pred, true_masks)
            acc_val +=dice( masks_pred, true_masks)
            val_loss += loss_val.item()

    batch_size
    print('Epoch :',t+1,'/',str(epoch_number))
    print('Validation Loss:v',(val_loss)/(val_size/batch_size))
    print('Validation Acc:',(acc_val)/(val_size/batch_size))
    print('Train Acc:',(acc_train)/(tra_size/batch_size))
    print('Train  Loss   :t',(train_loss)/(tra_size/batch_size))
    # Store epoch progress
    val_loss_array.append((val_loss)/(val_size/batch_size))
    train_loss_array.append((train_loss)/(tra_size/batch_size))
    acc_val_array.append((acc_val)/(val_size/batch_size))
    acc_train_array.append((acc_train)/(tra_size/batch_size))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3704: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


Epoch : 1 / 20
Validation Loss:v 0.06251861775914828
Validation Acc: 0.9794907520214716
Train Acc: 0.9257164613809437
Train  Loss   :t 0.13277385546825826
Epoch : 2 / 20
Validation Loss:v 0.04401507445921501
Validation Acc: 0.9834581166505814
Train Acc: 0.9832557335495948
Train  Loss   :t 0.04838066534139216
Epoch : 3 / 20
Validation Loss:v 0.03597574168816209
Validation Acc: 0.9856001188357671
Train Acc: 0.9868132017552853
Train  Loss   :t 0.03445241595618427
Epoch : 4 / 20
Validation Loss:v 0.03354007735227545
Validation Acc: 0.986124649643898
Train Acc: 0.9884492672979832
Train  Loss   :t 0.027482391521334647
Epoch : 5 / 20
Validation Loss:v 0.03289222437888384
Validation Acc: 0.9857103675603867
Train Acc: 0.989058343321085
Train  Loss   :t 0.024319621408358215
Epoch : 6 / 20
Validation Loss:v 0.02874436617518465
Validation Acc: 0.9869970281918844
Train Acc: 0.9889132536947727
Train  Loss   :t 0.023492059437558055
Epoch : 7 / 20
Validation Loss:v 0.02986158135657509
Validation Acc: 

In [ ]:
import copy
best_model_wts = copy.deepcopy(net.state_dict())
torch.save({    
    'epoch':40 + 1,
    'model_state_dict':best_model_wts,
    'optimizer_state_dict': optimizer.state_dict(),
    'best_acc': 0.18
},os.path.join(os.getcwd(),'/content/drive/Shareddrives/FurkanDataandModels/Furkan_covid/codes/lung_deepnet_{}_epo.pth'.format(20 + 1)))

In [ ]:
# Save loss and acc information to the txt 
textfile = open("/content/drive/Shareddrives/FurkanDataandModels/Furkan_covid/codes/lung_deepnet_{}_epo.txt".format(epoch_number), "w")
biglist=val_loss_array+   train_loss_array+    acc_val_array+    acc_train_array
textfile. write("running_loss_array+   train_loss_array+    acc_val_array+    acc_train_array"+ "\n")
for element in [biglist]:
    textfile. write(str(element) + "\n")
textfile. close()